In [1]:
import matplotlib.pyplot as plt
import pytesseract as pt
import os
import cv2
import re
import pandas as pd

In [32]:
pt.pytesseract.tesseract_cmd=r'C:\Program Files\Tesseract-OCR\tesseract.exe'
coln=['Registration Number','Registration Date','Chassis Number','Engine Number','Manufacturing Date','Name']
df=pd.DataFrame(columns=coln)

In [39]:
def load_images_from_folder(folder,alter=True,blur=False,resize=False,scale=100,w_stretch=0):
    
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        
        if resize==True:
            scale_percent = scale # percent of original size
            width = int(img.shape[1] * scale_percent / 100)+w_stretch
            height = int(img.shape[0] * scale_percent / 100)
            dim = (width, height)
            img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
            
        if alter==True:
            img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            img = cv2.threshold(img, 170, 200,cv2.THRESH_TRUNC)[1]
        
        if blur==True:
            img = cv2.GaussianBlur(img,(3,3),0)
            
        if img is not None:
            images.append(img)
            
        regex(images[-1])
    return #images

In [40]:
def regex(img):
    rc_data=pt.image_to_string(img)
    if re.search('Licence',rc_data) != None:
        return
    reg=re.search('\s[A-Z]{2}[0-9]{1}[A-Z]{2}\s[0-9]{4}|\s[A-Z]{2}[0-9]{2}[A-Z]{1}\s[0-9]{4}|\s[A-Z]{2}[0-9]{2}[A-Z]{2}[0-9]{4}|[A-Z]{2}[0-9]{2}-[A-Z]{1}-[0-9]{4}|\s[A-Z]{2}[0-9]{1}[A-Z]{3}[0-9]{4}',rc_data)
    reg_date=re.search('(?<=Issue).*[A-Za-z0-9\-]|(?<=Valid From).*[A-Za-z0-9\-][0-9]{4}\s|(?<=DT:).[0-9/]+',rc_data)
    chassis=re.search('[0-9A-Z]{11}[0-9]{6}|[0-9A-Z]{11}\s[0-9]{6}|[0-9A-Z]{17}\*[A-Z]|[0-9A-Z]{9}[0-9]{5}',rc_data)
    engine=re.search('[0-9A-Z]{6}[0-9]{6}\s|\s[0-9A-Z]{6}\s[0-9]{6}|\s[0-9A-Z]{5}\s[0-9]{7}|\s[0-9A-Za-z]{4}[0-9]{7}|\s[0-9A-Za-z]{4}-[0-9]{7}|(?<=NO)\.* :* [0-9]*[0-9]{6}',rc_data)
    mfg_dt=re.search('(?<=Manufacture).*[A-Z0-9a-z]|(?<=Mfg.).*[A-Z0-9a-z]|(?<=MFG)\.*[A-Z]{2}\.*:* [0-9]{2}/[0-9]{4}',rc_data)
    name=re.search('(?<=Name )[A-Z]+|(?<=NAME ):*[A-Z]+|(?<=Address ):*[A-Z]+',rc_data)
    refine(reg,reg_date,chassis,engine,mfg_dt,name)

In [41]:
def refine(r,rd,c,e,m,n):
    col=[]
    global df
    if r==None:
        r=['NA']
    col.append(r[0])
    if rd==None:
        rd=['NA']
    col.append(rd[0])
    if c==None:
        c=['NA']
    col.append(c[0])
    if e==None:
        e=['NA']
    col.append(e[0])
    if m==None:
        m=['NA']
    m=re.search('[0-9]{2}.*|NA',m[0])
    col.append(m[0])
    if n==None:
        n=['NA']
    n=re.search('[A-Z].*|NA',n[0])
    col.append(n[0])
    if r[0] not in list(df['Registration Number']):
        d=dict(zip(coln,col))
        df=df.append(d,ignore_index=True)
    return

In [42]:
load_images_from_folder('C:/Users/lucif/Documents/RC PROJECT/RC')

In [43]:
df

,Registration Number,Registration Date,Chassis Number,Engine Number,Manufacturing Date,Name
0,DL13CA8614,11/02/2014,MA3FHEB1S00593004,EB1S00593004,01/2014,RAKESH KUMAR
1,DL9CAC6215,NA,NA,NA,NA,NA
2,DL2CAT9109,NA,MA3ETDE1S00218363,DE1S00218363\n,NA,ANOOP
3,NA,07-Feb-2007,MA3EYD81S00765439,D81S00765439,912006,SUBE
4,MA3EYD8180,a3-Jan-2008,MA3EYD81801277497,D81801277497,NA,RANBEER
5,HR49D 0002,3-Nov-2013,NA,013A2235550,912013,AMAR
6,SO0ST 2865,NA,NA,NA,NA,NA
7,DL2CAU7997,NA,MA3FLEB1S00309631,EB1S00309631,NA,NA
8,DL3CBU4111,03/08/2012,MA3FDEB1200381622,EB1200381622,NA,NA
9,DL9CAC1454,NA,MA3EADE1S00237286,DE1S00237286,NA,KUMUDHA
